In [471]:
import pandas as pd
import numpy as np

In [472]:

df = pd.read_csv("number_to_word_1_99.csv")

In [473]:
df.head()

,number,word
0,1,one
1,2,two
2,3,three
3,4,four
4,5,five


In [474]:
X = df["number"].tolist()
y = df["word"].tolist()

In [475]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
tokenizer_in = Tokenizer()
tokenizer_out = Tokenizer(lower=True)

In [476]:
X = [str(num) for num in X]
tokenizer_in.fit_on_texts(X)
X = tokenizer_in.texts_to_sequences(X)
x_max_len = max(len(a) for a in X)
X = pad_sequences(X, maxlen=x_max_len,padding='post')

In [477]:
y = ["<start> " + word + " <end>" for word in y]
tokenizer_out.fit_on_texts(y)
y = tokenizer_out.texts_to_sequences(y)
y_max_len = max(len(a) for a in y)
y = pad_sequences(y, maxlen=y_max_len+1,padding='post')


In [478]:
y_input = y[:, :-1]
y_target = y[:, 1:]

In [479]:
vacab_out = len(tokenizer_out.word_index)+1
y_target = to_categorical(y_target)


In [480]:
vacab_in = len(tokenizer_in.word_index)+1
vacab_in

100

In [481]:
vacab_out

30

In [482]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,LSTM,Dense,Input

In [483]:
encoder_in = Input(shape=(x_max_len,))
encoder_emd_layer = Embedding(vacab_in,10)
encoder_emd = encoder_emd_layer(encoder_in)
encoder_lstm = LSTM(10,return_state=True)
_,h,c = encoder_lstm(encoder_emd)

decoder_in = Input(shape=(y_max_len,))
decoder_emd_layer = Embedding(vacab_out,10)
decoder_emd = decoder_emd_layer(decoder_in)
decoder_lstm = LSTM(10,return_sequences=True,return_state=True)
decoder_out,_,_ = decoder_lstm(decoder_emd,initial_state=[h,c])

dense = Dense(vacab_out,activation="softmax")
output = dense(decoder_out)

In [484]:
model = Model([encoder_in,decoder_in],output)

In [485]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [521]:
history = model.fit(
    [X,y_input],
    y_target,
    epochs=50,
    verbose=1
)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9823 - loss: 0.2185
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9823 - loss: 0.2178
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9823 - loss: 0.2172
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9823 - loss: 0.2166
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9823 - loss: 0.2159
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9823 - loss: 0.2152
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9823 - loss: 0.2146
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9823 - loss: 0.2139
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9823 - loss: 0.2133
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9823 - loss: 0.2127
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9823 - loss: 0.2121
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9823 - loss: 0.2115
E

In [523]:
import numpy as np

# Example: your tokenizer's word_index
tokenizer_word_index = {
    'start': 1, 'end': 2, 'twenty': 3, 'thirty': 4, 'forty': 5, 'fifty': 6,
    'sixty': 7, 'seventy': 8, 'eighty': 9, 'ninety': 10, 'one': 11, 'two': 12,
    'three': 13, 'four': 14, 'five': 15, 'six': 16, 'seven': 17, 'eight': 18,
    'nine': 19, 'ten': 20, 'eleven': 21, 'twelve': 22, 'thirteen': 23,
    'fourteen': 24, 'fifteen': 25, 'sixteen': 26, 'seventeen': 27,
    'eighteen': 28, 'nineteen': 29
}

# Inverse dictionary: index -> word
idx2word = {v: k for k, v in tokenizer_word_index.items()}

# Tokens
start_token = tokenizer_word_index['start']
end_token = tokenizer_word_index['end']

# Input number
data = 57
encoder_input = np.array([[data]])  # shape = (1, 1)

# Fixed decoder length (same as trained model)
decoder_length = 4
decoder_input = np.zeros((1, decoder_length), dtype=int)
decoder_input[0, 0] = start_token

decoded_sequence = []

for t in range(1, decoder_length):
    # Predict next token
    preds = model.predict([encoder_input, decoder_input], verbose=0)
    # preds shape: (1, timesteps, vocab_size)
    next_token = np.argmax(preds[0, t-1, :])

    if next_token == end_token:
        break

    decoded_sequence.append(next_token)
    decoder_input[0, t] = next_token  # feed predicted token to next timestep

# Convert tokens to words
decoded_words = [idx2word[i] for i in decoded_sequence]
print("Number in words:", " ".join(decoded_words))


Number in words: fifty seven
